In [2]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, KEYWORD
from whoosh.qparser import QueryParser
import os
import difflib

In [3]:
# 인덱스 디렉토리 생성
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

# 스키마 정의
schema = Schema(
    incomplete_disease_code=KEYWORD(stored=True),
    incomplete_disease_name=TEXT(stored=True),
    claim_category=TEXT(stored=True),
    claim_detail=TEXT(stored=True),
)

In [4]:
# 인덱스 생성
ix = create_in("indexdir", schema)

# 데이터 색인
writer = ix.writer()
data = {
    "incomplete_disease_code": [
        "K00.01",
        "K00.12",
        "K01.0",
        "K02.3",
        "K02.8",
        "K03.01",
        "K03.5",
        "K04.00",
        "K05.08",
        "K05.13",
        "K06.01",
        "K06.19",
        "K07.66",
        "K08.81",
        "S01.41",
        "S03.22",
        "B00.2",
        "R12",
        "T85.6",
        "Z46.3",
    ],
    "incomplete_disease_name": [
        "완전무치증",
        "대구치 부위 과잉치",
        "매몰치",
        "정지된 치아 우식",
        "기타 치아우식",
        "인접면 마모",
        "치아의 강직증",
        "가역적 치수염",
        "기타 명시된 급성 치은염",
        "만성 박리성 치은염",
        "전반적 치은퇴축",
        "상세불명의 치은비대",
        "저작근장애",
        "불규칙치조돌기",
        "상악부의 열린상처",
        "치아의 아탈구 (측방 탈구)",
        "헤르페스바이러스 치은구내염 및 인두편도염",
        "속쓰림",
        "기타 명시된 내부 인공삽입장치, 삽입물 및 이식편의 기계적 합병증",
        "치과보철 장치의 부착 및 조정",
    ],
    "claim_category": [
        "",
        "외과",
        "외과",
        "청구 기초 및 기본진료, 보존",
        "청구 기초 및 기본진료, 보존, 외과",
        "청구 기초 및 기본진료",
        "외과",
        "청구 기초 및 기본진료",
        "보존, 치주",
        "청구 기초 및 기본진료, 치주",
        "보존, 치주",
        "외과",
        "청구 기초 및 기본진료, 턱관절",
        "외과",
        "",
        "보존, 근관치료, 치주, 외과",
        "청구 기초 및 기본진료",
        "",
        "청구 기초 및 기본진료, 보존, 근관치료, 임플란트",
        "보존, 임플란트, 틀니",
    ],
    "claim_detail": [
        "",
        "과잉치 발치",
        "단순매복, 복잡매복, 완전매복",
        "기본진료 종류 및 상병, 상세불명 vs 기타 명시된 상병명 차이, 광중합형 복합레진 충전, 보철물 제거 (복잡), 보철물제거 간단, 보통처치, 수복물 제거 (간단), 즉일충전처치, 진정처치, 충전물 연마, 치수복조, 유치 발치, 치관확장술",
        "기본진료 종류 및 상병, 상세불명 vs 기타 명시된 상병명 차이, 광중합형 복합레진 충전, 보철물 제거 (복잡), 보철물제거 간단, 보통처치, 수복물 제거 (간단), 즉일충전처치, 진정처치, 충전물 연마, 치수복조, 유치 발치, 치관확장술",
        "기본진료 종류 및 상병",
        "난발치, 유치 발치",
        "기본진료 종류 및 상병, 치근단, 치근단 동시 촬영",
        "치면세마, 치주치료 후처치",
        "기본진료 종류 및 상병, 기본진료시 항생제 처방 상병, 치석제거 (가), 치석제거 (가) vs (나), 치석제거 (나) = 연 회, 치주치료 후처치",
        "지각과민처치 (간단), 지각과민처치 (복잡), 치은이식술",
        "치관확장술, 치은절제술",
        "기본진료 종류 및 상병, 기본진료시 항생제 처방 상병, 분사신장치료, 악관절자극요법 (단순, 전기, 복합), 파노라마 촬영 특수",
        "치조골 성형술",
        "",
        "교합 조정, 근관세척, 근관충전, 근관확대, 근관성형, 발수, 근관와동형성, 신경치료 상병명, 잠간고정술, 고정장치 제거, 탈구치아정복술",
        "구내염처치, 단순염증 처치",
        "",
        "Bur, 보철물 재부착, 보철물 제거 (복잡), 보철물제거 간단, 수복물 제거 (간단), 충전처치, 임플란트 홀 충전, 파절기구제거, 임플란트 유지관리, 임플란트 제거술",
        "보철물 재부착, 보철물 제거 (복잡), 보철물제거 간단, 수복물 제거 (간단), 임플란트 유지관리, 부분틀니, 완전틀니, 틀니 유지관리, 의치 조직면 개조, 의치 수리, 의치 조정, 클라스프 수리",
    ],
}

# 색인 데이터 추가
for code, name, category, detail in zip(
    data["incomplete_disease_code"],
    data["incomplete_disease_name"],
    data["claim_category"],
    data["claim_detail"],
):
    writer.add_document(
        incomplete_disease_code=code,
        incomplete_disease_name=name,
        claim_category=(
            category if category else "상품 없음"
        ),  # 빈 문자열을 '상품 없음'으로 처리
        claim_detail=(
            detail if detail else "상세 정보 없음"
        ),  # 빈 문자열을 '상세 정보 없음'으로 처리
    )

writer.commit()

In [5]:
# 오타 교정 기능
def correct_spelling(query):
    all_diseases = data["incomplete_disease_name"]  # 모든 병명 데이터를 불러옴
    matches = difflib.get_close_matches(
        query, all_diseases, n=1, cutoff=0.6
    )  # 유사한 단어 찾기
    return (
        matches[0] if matches else query
    )  # 가장 가까운 단어 반환, 없으면 원래 검색어 반환

# 검색 함수
def search_claims(natural_query):

    # 오타 교정
    corrected_query = correct_spelling(natural_query)

    # 여러 키워드 처리
    keywords = corrected_query.split()
    queries = [f"{keyword}*" for keyword in keywords]  # 부분 일치 검색 (모든 단어 포함)

    with ix.searcher() as searcher:
        # 각 키워드를 OR 조건으로 결합
        query_string = " OR ".join(queries)
        query = QueryParser("incomplete_disease_name", ix.schema).parse(query_string)
        results = searcher.search(query)

        # 결과 가중치 조정 (상위 5개 결과만 반환)
        ranked_results = sorted(results, key=lambda r: r.score, reverse=True)[:5]
        return [
            {
                "code": result["incomplete_disease_code"],
                "name": result["incomplete_disease_name"],
                "category": result["claim_category"],
                "detail": result["claim_detail"],
            }
            for result in ranked_results
        ]

In [6]:
queries = [
    # 정상적인 검색
    "완전무치증 치료",
    "기타 치아우식",
    "대구치 발치",
    # 오타 포함
    "워전무치즁",  # 오타: '완전무치증'
    "대구티 부의 발치",  # 오타: '대구치 부위 발치'
    "상악부의 열림 상처",  # 오타: '상악부의 열린상처'
    # 영문 자판으로 잘못 입력된 한글
    "dkstn",  # 잘못된 입력: '완전'
    "rmfwlcjs",  # 잘못된 입력: '치은염'
    "qudmlqnx",  # 잘못된 입력: '과잉치 발치'
    "xmftls dnlwlcjs",  # 잘못된 입력: '보존 치료'
    # 일상 문장
    "치아를 뽑고 싶어요",
    "이빨이 너무 아파요",
    "뭔가 잇몸이 부어올랐어요",
    # 다중 단어 쿼리
    "상악부 열린 상처 치료",
    "매복치 발치와 보철물 제거",
    "기타 명시된 급성 치은염 치료",
    # 자판 혼합
    "gkrwjddml dkwjs",  # 잘못된 입력: '급성의 치료'
    "치은m 치료",  # 혼합된 입력: '치은염 치료'
    # 복잡한 오타 및 잘못된 입력
    "tkwrhd tnws",  # 잘못된 입력: '치료 처리'
    "clrqnfwh chlrwpcl",  # 잘못된 입력: '과잉치 발치'
    # 코드로만 검색
    "K00.01",  # 완전무치증
    "K02.8",  # 기타 치아우식
    "K05.08",  # 기타 명시된 급성 치은염
    "S01.41",  # 상악부의 열린상처
    "Z46.3",  # 치과보철 장치의 부착 및 조정
]

In [7]:
# 각 쿼리로 검색 수행
for query in queries:
    results = search_claims(query)
    print(f"Query: {query}")
    if results:
        for result in results:
            print(
                f"Code: {result['code']}, Name: {result['name']}, Category: {result['category']}, Detail: {result['detail']}"
            )
    else:
        print("No results found.")
    print("===")

Query: 완전무치증 치료
Code: K00.01, Name: 완전무치증, Category: 상품 없음, Detail: 상세 정보 없음
===
Query: 기타 치아우식
Code: K02.8, Name: 기타 치아우식, Category: 청구 기초 및 기본진료, 보존, 외과, Detail: 기본진료 종류 및 상병, 상세불명 vs 기타 명시된 상병명 차이, 광중합형 복합레진 충전, 보철물 제거 (복잡), 보철물제거 간단, 보통처치, 수복물 제거 (간단), 즉일충전처치, 진정처치, 충전물 연마, 치수복조, 유치 발치, 치관확장술
Code: K05.08, Name: 기타 명시된 급성 치은염, Category: 보존, 치주, Detail: 치면세마, 치주치료 후처치
Code: T85.6, Name: 기타 명시된 내부 인공삽입장치, 삽입물 및 이식편의 기계적 합병증, Category: 청구 기초 및 기본진료, 보존, 근관치료, 임플란트, Detail: Bur, 보철물 재부착, 보철물 제거 (복잡), 보철물제거 간단, 수복물 제거 (간단), 충전처치, 임플란트 홀 충전, 파절기구제거, 임플란트 유지관리, 임플란트 제거술
===
Query: 대구치 발치
Code: K00.12, Name: 대구치 부위 과잉치, Category: 외과, Detail: 과잉치 발치
===
Query: 워전무치즁
Code: K00.01, Name: 완전무치증, Category: 상품 없음, Detail: 상세 정보 없음
===
Query: 대구티 부의 발치
Code: K00.12, Name: 대구치 부위 과잉치, Category: 외과, Detail: 과잉치 발치
===
Query: 상악부의 열림 상처
Code: S01.41, Name: 상악부의 열린상처, Category: 상품 없음, Detail: 상세 정보 없음
===
Query: dkstn
No results found.
===
Query: rmfwlcjs
No results found.
===
Query: qudmlqnx
No resu